<a href="https://colab.research.google.com/github/Fermu25/Cursos/blob/main/Comprimir%20PDF(Dividir%2Cfiltrar%2Ccomprimir%2Cunir).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.1 MB/s eta 0:00:00


In [ ]:
pip install pikepdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 68.1 MB/s eta 0:00:00


In [ ]:
import platform
import shutil

def has_cmd(cmd: str) -> bool:
    return shutil.which(cmd) is not None

def _gs_executable():
    if platform.system().lower().startswith("win"):
        return "gswin64c"
    return "gs"

gs_cmd = _gs_executable()
if has_cmd(gs_cmd):
    print(f"Ghostscript ('{gs_cmd}') is installed and found in the PATH.")
else:
    print(f"Ghostscript ('{gs_cmd}') was not found in the PATH.")
    print("Please ensure Ghostscript is installed and its executable is in your system's PATH.")

Ghostscript ('gs') was not found in the PATH.
Please ensure Ghostscript is installed and its executable is in your system's PATH.


In [ ]:
pip install pymupdf pikepdf


In [ ]:
import os
import sys
import shutil
import platform
from pathlib import Path

import fitz
import pikepdf

INPUT_FILE   = "fisco agenda 2025.pdf"
OUTPUT_FILE  = "salida.pdf"
TEMP_DIR     = Path("temp_pdf")
BLOCK_SIZE   = 200
REPAIRED_PDF = TEMP_DIR / "repaired.pdf"

TARGET_DPI       = 150
JPEG_QUALITY     = 85
COLOR_MODE       = "gray"
CLEAN_GARBAGE    = 4

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def repair_with_pikepdf(src: Path, dst: Path) -> bool:
    try:
        with pikepdf.open(str(src)) as pdf:
            pdf.save(str(dst))
        return dst.exists() and dst.stat().st_size > 0
    except Exception:
        return False

def normalize_pdf(input_path: Path, repaired_path: Path) -> Path:
    ok = repair_with_pikepdf(input_path, repaired_path)
    if ok:
        return repaired_path
    return input_path

def split_with_fitz(src: Path, temp_dir: Path, block_size: int):
    parts = []
    with fitz.open(str(src)) as doc:
        total = doc.page_count
        for start in range(0, total, block_size):
            end = min(start + block_size, total)
            part_doc = fitz.open()
            part_doc.insert_pdf(doc, from_page=start, to_page=end - 1)
            part_name = temp_dir / f"part_{len(parts)+1}.pdf"
            part_doc.save(str(part_name), garbage=CLEAN_GARBAGE, deflate=True, clean=True)
            part_doc.close()
            parts.append(part_name)
    return parts

def compress_block_by_rasterizing(input_pdf: Path, output_pdf: Path,
                                  dpi: int = 150, jpg_quality: int = 85,
                                  color_mode: str = "rgb"):
    zoom = dpi / 72.0
    mat = fitz.Matrix(zoom, zoom)
    if color_mode.lower() == "gray":
        cs = fitz.csGRAY
    else:
        cs = fitz.csRGB
    with fitz.open(str(input_pdf)) as src:
        out = fitz.open()
        total = src.page_count
        for i in range(total):
            page = src[i]
            pix = page.get_pixmap(matrix=mat, colorspace=cs, alpha=False)
            img_bytes = pix.tobytes(output="jpeg", jpg_quality=jpg_quality)
            rect = fitz.Rect(0, 0, pix.width, pix.height)
            new_page = out.new_page(width=rect.width, height=rect.height)
            new_page.insert_image(rect, stream=img_bytes)
        out.save(str(output_pdf), garbage=CLEAN_GARBAGE, deflate=True, clean=True)
        out.close()

def merge_with_fitz(parts, output_pdf: Path):
    out = fitz.open()
    for p in parts:
        with fitz.open(str(p)) as part_doc:
            out.insert_pdf(part_doc)
    out.save(str(output_pdf), garbage=CLEAN_GARBAGE, deflate=True, clean=True)
    out.close()

def safe_remove(path: Path):
    try:
        if path.is_dir():
            shutil.rmtree(path)
        elif path.exists():
            path.unlink()
    except Exception:
        pass

def main():
    in_path = Path(INPUT_FILE)
    if not in_path.exists():
        sys.exit(1)
    ensure_dir(TEMP_DIR)
    normalized = normalize_pdf(in_path, REPAIRED_PDF)
    parts = split_with_fitz(normalized, TEMP_DIR, BLOCK_SIZE)
    compressed_parts = []
    for part in parts:
        comp = part.with_stem(part.stem + f"_cmp_{TARGET_DPI}dpi_q{JPEG_QUALITY}")
        compress_block_by_rasterizing(
            input_pdf=part,
            output_pdf=comp,
            dpi=TARGET_DPI,
            jpg_quality=JPEG_QUALITY,
            color_mode=COLOR_MODE
        )
        compressed_parts.append(comp)
    merge_with_fitz(compressed_parts, Path(OUTPUT_FILE))

if __name__ == "__main__":
    main()


In [ ]:
import os
import sys
import shutil
import platform
from pathlib import Path

import fitz  # PyMuPDF
import pikepdf

# ========= CONFIG =========
INPUT_FILE   = "fisco agenda 2025.pdf"   # <-- tu PDF original (con espacios)
OUTPUT_FILE  = "salida.pdf"              # PDF final comprimido
TEMP_DIR     = Path("temp_pdf")
BLOCK_SIZE   = 200                       # páginas por bloque (ajústalo si tu RAM es limitada)
REPAIRED_PDF = TEMP_DIR / "repaired.pdf"

# Parámetros de compresión internos (SIN Ghostscript)
TARGET_DPI       = 150     # 150–200 suele ser buen equilibrio para lectura
JPEG_QUALITY     = 85      # 60–90; 80–85 da muy buen ahorro con texto legible
COLOR_MODE       = "rgb"   # "rgb" o "gray" (gris reduce más peso, pero cambia color)
# LINEAR_SAVE      = True    # lineariza PDF para abrir más rápido - Linearisation is no longer supported
CLEAN_GARBAGE    = 4       # limpieza de objetos (0..4). 4 = máxima

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

# ========= REPARAR / NORMALIZAR =========
def repair_with_pikepdf(src: Path, dst: Path) -> bool:
    """
    Repara/normaliza reescribiendo el archivo con pikepdf.
    Suele arreglar tablas XRef raras y metadatos problemáticos.
    """
    try:
        with pikepdf.open(str(src)) as pdf:
            # 'Stream' re-writes often help; default save is fine
            pdf.save(str(dst))
        return dst.exists() and dst.stat().st_size > 0
    except Exception:
        return False

def normalize_pdf(input_path: Path, repaired_path: Path) -> Path:
    """
    Normaliza el PDF: pikepdf -> fallback al original si no se pudo.
    """
    print("🔧 Intentando reparar/normalizar el PDF con pikepdf...")
    ok = repair_with_pikepdf(input_path, repaired_path)
    if ok:
        print("✅ Normalizado con pikepdf.")
        return repaired_path
    print("⚠️ No fue posible normalizar; seguiremos con el archivo original.")
    return input_path

# ========= DIVIDIR (con PyMuPDF, SIN rasterizar) =========
def split_with_fitz(src: Path, temp_dir: Path, block_size: int):
    """
    Divide el PDF en partes usando PyMuPDF insertando rangos de páginas.
    No rasteriza; copia páginas nativas (rápido y robusto).
    """
    print("📂 Abriendo PDF para dividir (PyMuPDF)...")
    parts = []
    with fitz.open(str(src)) as doc:
        total = doc.page_count
        print(f"📄 Páginas detectadas: {total}")

        for start in range(0, total, block_size):
            end = min(start + block_size, total)  # end no-inclusivo
            # Crear doc destino y copiar rango [start, end)
            part_doc = fitz.open()
            part_doc.insert_pdf(doc, from_page=start, to_page=end - 1)
            part_name = temp_dir / f"part_{len(parts)+1}.pdf"
            part_doc.save(str(part_name), garbage=CLEAN_GARBAGE, deflate=True, clean=True) # Removed linear=LINEAR_SAVE
            part_doc.close()
            parts.append(part_name)
            print(f"✅ Bloque guardado: {part_name} (páginas {start+1}-{end})")
    return parts

# ========= COMPRESIÓN SIN GS (rasterizar a JPEG con DPI y calidad) =========
def compress_block_by_rasterizing(input_pdf: Path, output_pdf: Path,
                                  dpi: int = 150, jpg_quality: int = 85,
                                  color_mode: str = "rgb"):
    """
    Abre el bloque, renderiza cada página a un pixmap según DPI deseado
    y crea un nuevo PDF insertando cada página como imagen JPEG.
    Esto reduce drásticamente PDFs escaneados manteniendo legibilidad.
    """
    zoom = dpi / 72.0  # 72 dpi es el “base” de PDF
    mat = fitz.Matrix(zoom, zoom)

    # Elegir espacio de color
    if color_mode.lower() == "gray":
        cs = fitz.csGRAY
    else:
        cs = fitz.csRGB

    with fitz.open(str(input_pdf)) as src:
        out = fitz.open()
        total = src.page_count

        for i in range(total):
            page = src[i]
            # Render en el color space elegido (sin alfa)
            pix = page.get_pixmap(matrix=mat, colorspace=cs, alpha=False)

            # Convertir a JPEG en memoria con la calidad deseada
            img_bytes = pix.tobytes(output="jpeg", jpg_quality=jpg_quality)

            # Crear página del tamaño exacto de la imagen renderizada
            rect = fitz.Rect(0, 0, pix.width, pix.height)
            new_page = out.new_page(width=rect.width, height=rect.height)

            # Insertar la imagen JPEG como stream (no PNG, para ahorrar espacio)
            new_page.insert_image(rect, stream=img_bytes)

            if (i + 1) % 20 == 0 or (i + 1) == total:
                print(f"   ⏳ Renderizadas {i+1}/{total} páginas en {input_pdf.name}")

        # Guardar con limpieza y compresión de streams
        out.save(str(output_pdf),
                 garbage=CLEAN_GARBAGE,
                 deflate=True,
                 clean=True) # Removed linear=LINEAR_SAVE
        out.close()

# ========= UNIR (con PyMuPDF, SIN rasterizar) =========
def merge_with_fitz(parts, output_pdf: Path):
    """
    Une PDFs con PyMuPDF sin rasterizar (solo concatena).
    """
    print("📚 Uniendo partes comprimidas...")
    out = fitz.open()
    for p in parts:
        with fitz.open(str(p)) as part_doc:
            out.insert_pdf(part_doc)
    out.save(str(output_pdf), garbage=CLEAN_GARBAGE, deflate=True, clean=True) # Removed linear=LINEAR_SAVE
    out.close()
    print(f"📕 PDF final creado: {output_pdf}")

# ========= LIMPIEZA OPCIONAL =========
def safe_remove(path: Path):
    try:
        if path.is_dir():
            shutil.rmtree(path)
        elif path.exists():
            path.unlink()
    except Exception:
        pass

# ========= MAIN =========
def main():
    in_path = Path(INPUT_FILE)
    if not in_path.exists():
        print(f"❌ No se encontró: {in_path.resolve()}")
        sys.exit(1)

    ensure_dir(TEMP_DIR)

    # 1) Reparar/normalizar (no obligatorio, pero recomendable)
    normalized = normalize_pdf(in_path, REPAIRED_PDF)

    # 2) Dividir (sin rasterizar)
    print("📦 Dividiendo PDF en bloques...")
    parts = split_with_fitz(normalized, TEMP_DIR, BLOCK_SIZE)

    # 3) Comprimir cada bloque SIN Ghostscript (rasterizando a JPEG)
    compressed_parts = []
    for part in parts:
        comp = part.with_stem(part.stem + f"_cmp_{TARGET_DPI}dpi_q{JPEG_QUALITY}")
        print(f"🔄 Comprimiendo (sin GS) {part.name} ...")
        compress_block_by_rasterizing(
            input_pdf=part,
            output_pdf=comp,
            dpi=TARGET_DPI,
            jpg_quality=JPEG_QUALITY,
            color_mode=COLOR_MODE
        )
        compressed_parts.append(comp)

    # 4) Unir bloques comprimidos
    merge_with_fitz(compressed_parts, Path(OUTPUT_FILE))

    print("✨ Listo. PDF comprimido generado.")
    print(f"👉 Archivo final: {OUTPUT_FILE}")
    print(f"🔎 Parámetros: DPI={TARGET_DPI}, JPEG_QUALITY={JPEG_QUALITY}, COLOR_MODE={COLOR_MODE}")

if __name__ == "__main__":
    main()

🔧 Intentando reparar/normalizar el PDF con pikepdf...
✅ Normalizado con pikepdf.
📦 Dividiendo PDF en bloques...
📂 Abriendo PDF para dividir (PyMuPDF)...
📄 Páginas detectadas: 1566
✅ Bloque guardado: temp_pdf/part_1.pdf (páginas 1-200)
✅ Bloque guardado: temp_pdf/part_2.pdf (páginas 201-400)
✅ Bloque guardado: temp_pdf/part_3.pdf (páginas 401-600)
✅ Bloque guardado: temp_pdf/part_4.pdf (páginas 601-800)
✅ Bloque guardado: temp_pdf/part_5.pdf (páginas 801-1000)
✅ Bloque guardado: temp_pdf/part_6.pdf (páginas 1001-1200)
✅ Bloque guardado: temp_pdf/part_7.pdf (páginas 1201-1400)
✅ Bloque guardado: temp_pdf/part_8.pdf (páginas 1401-1566)
🔄 Comprimiendo (sin GS) part_1.pdf ...
   ⏳ Renderizadas 20/200 páginas en part_1.pdf
   ⏳ Renderizadas 40/200 páginas en part_1.pdf
   ⏳ Renderizadas 60/200 páginas en part_1.pdf
   ⏳ Renderizadas 80/200 páginas en part_1.pdf
   ⏳ Renderizadas 100/200 páginas en part_1.pdf
   ⏳ Renderizadas 120/200 páginas en part_1.pdf
   ⏳ Renderizadas 140/200 páginas en